# **Hello Credit 0.0.3 (Model Demo)**


# NOTES:

* Returns are more important than the methodology
* Backtest against Moody's, Bloomberg Models
* "Do not let, perfect stand in the way of good"


In [1]:
import numpy as np
import pandas as pd

In [2]:
import pandas as pd

# Initial data
data = {
    "Company": ["Company A", "Company B", "Company C", "Company D", "Company E", "Company F"],
    "Return_on_Assets": [0.02, 0.04, 0.07, 0.03, 0.05, 0.09],
    "Operating_Margin_Ratio": [0.03, 0.10, 0.20, 0.05, 0.12, 0.25],
    "Net_Profit_Margin": [0.02, 0.06, 0.12, 0.03, 0.07, 0.15],
    "Debt_to_EBITDA": [5.0, 3.5, 1.5, 4.0, 2.5, 1.0],
    "EBITDA_to_Interest_Expense": [1.2, 2.0, 3.5, 1.5, 2.5, 4.0],
    "Asset_Turnover": [0.4, 0.8, 1.6, 0.5, 1.0, 2.0]
}
df = pd.DataFrame(data)

In [3]:
# Classification values and ranges
classification_values = {'Poor': 1, 'Fair': 2, 'Good': 3}


classification_ranges = {
    'Return_on_Assets': {'Poor': (None, 0.02), 'Fair': (0.02, 0.05), 'Good': (0.05, None)},
    'Operating_Margin_Ratio': {'Poor': (None, 0.10), 'Fair': (0.10, 0.20), 'Good': (0.20, None)},
    'Net_Profit_Margin': {'Poor': (None, 0.05), 'Fair': (0.05, 0.10), 'Good': (0.10, None)},
    'Debt_to_EBITDA': {'Good': (None, 2.0), 'Fair': (2.0, 4.0), 'Poor': (4.0, None)},  # Note: Inverted logic for Debt to EBITDA
    'EBITDA_to_Interest_Expense': {'Poor': (None, 2.0), 'Fair': (2.0, 4.0), 'Good': (4.0, None)},
    'Asset_Turnover': {'Poor': (None, 0.5), 'Fair': (0.5, 1.5), 'Good': (1.5, None)}
}

# Function to classify metric values into "Poor", "Fair", and "Good"
def classify_metric(value, metric):
    ranges = classification_ranges[metric]
    for classification, (low, high) in ranges.items():
        if low is None and value <= high:
            return classification
        elif high is None and value > low:
            return classification
        elif low is not None and high is not None and low <= value <= high:
            return classification
    return "Unknown"

# Apply classification for each metric
for metric in classification_ranges.keys():
    df[f'{metric}_Class'] = df[metric].apply(lambda x: classify_metric(x, metric))

# Function to calculate overall score and classification
def calculate_and_classify_overall(row):
    scores = [classification_values[row[f'{metric}_Class']] for metric in classification_ranges]
    overall_score = sum(scores) / len(scores)
    overall_class = 'Good' if overall_score > 2.5 else 'Fair' if overall_score > 1.5 else 'Poor'
    return pd.Series([overall_score, overall_class], index=['Overall_Score', 'Overall_Class'])

# Calculate overall scores and classifications
df[['Overall_Score', 'Overall_Class']] =df.apply(calculate_and_classify_overall, axis=1)

In [4]:
df.set_index('Company', inplace=True)

## Actual Metrics (Financial Ratios)

In [5]:
df.iloc[: , :6]

,Return_on_Assets,Operating_Margin_Ratio,Net_Profit_Margin,Debt_to_EBITDA,EBITDA_to_Interest_Expense,Asset_Turnover
Company,,,,,,
Company A,0.02,0.03,0.02,5.0,1.2,0.4
Company B,0.04,0.10,0.06,3.5,2.0,0.8
Company C,0.07,0.20,0.12,1.5,3.5,1.6
Company D,0.03,0.05,0.03,4.0,1.5,0.5
Company E,0.05,0.12,0.07,2.5,2.5,1.0
Company F,0.09,0.25,0.15,1.0,4.0,2.0


## Metric Classifications

In [6]:
df.iloc[: , 6:12]

,Return_on_Assets_Class,Operating_Margin_Ratio_Class,Net_Profit_Margin_Class,Debt_to_EBITDA_Class,EBITDA_to_Interest_Expense_Class,Asset_Turnover_Class
Company,,,,,,
Company A,Poor,Poor,Poor,Poor,Poor,Poor
Company B,Fair,Poor,Fair,Fair,Poor,Fair
Company C,Good,Fair,Good,Good,Fair,Good
Company D,Fair,Poor,Poor,Fair,Poor,Poor
Company E,Fair,Fair,Fair,Fair,Fair,Fair
Company F,Good,Good,Good,Good,Fair,Good


## Overall Scores and Classifications

In [7]:
df.iloc[: , 12:]

,Overall_Score,Overall_Class
Company,,
Company A,1.000000,Poor
Company B,1.666667,Fair
Company C,2.666667,Good
Company D,1.333333,Poor
Company E,2.000000,Fair
Company F,2.833333,Good
